# Building Multi-Agent AI Systems with ADK

| # | Section | Type |
|---|---------|------|
| 1 | What Are AI Agents? | Concepts |
| 2 | Single Agent to Multi-Agent | Concepts |
| 3 | Demo 1: Multi-Agent Routing | Live Code |
| 4 | Demo 2: MCP (Database Access) | Script |
| 5 | Demo 3: A2A + Full System | Script |
| 6 | Homework | Assignment |


---
# Section 1: What Are AI Agents?

A **chatbot** takes input, returns output. One turn.

An **agent** can: **Reason**, **Use tools**, **Remember context**, **Decide when done**.

Agent = LLM + Loop: **think -> act -> observe -> repeat**

### Agent Components

| Component | What It Does | Example |
|-----------|-------------|--------|
| **Instructions** | System prompt | "You are a billing specialist..." |
| **Model** | LLM engine | Gemini, GPT, Claude |
| **Tools** | Functions it can call | Query database, send email |
| **Memory** | What it remembers | Conversation history |

### The Agent Loop

```
User Input -> REASON (LLM) -> ACT (Tools) -> OBSERVE (Results) -> Done? -> Final Response
                                                                    No -> back to REASON
```


---
# Section 2: From Single Agent to Multi-Agent

### The Problem: "God Agent"
One agent with 15+ tools, a 2000-word prompt, contradicting instructions. Tool selection degrades.

### The Solution: Split the Work

```
        Root Agent (Router)
          /     |      \
    Billing  Technical  Escalation
     Agent     Agent      Agent
```

Each agent: focused job, short instructions, limited tools.

| Benefit | Description |
|---------|-------------|
| Modularity | Change one without breaking others |
| Specialization | Fewer tools = better tool selection |
| Testability | Test each agent in isolation |
| Scalability | Add agents, don't bloat existing ones |

### Multi-Agent Patterns

1. **Router / Delegation** -- Route to the right specialist
2. **Sequential Pipeline** -- Agents in order (Draft -> Review -> Edit)
3. **Parallel Fan-Out** -- Multiple agents simultaneously
4. **Loop / Iterative Refinement** -- Repeat until quality met


---
# Section 3: ADK -- Demo 1

**ADK (Agent Development Kit)** -- Google's open-source, model-agnostic framework for AI agents.

| Concept | ADK Implementation |
|---------|-------------------|
| Agent with tools | `Agent(instruction, tools, model)` |
| Multi-agent hierarchy | `sub_agents` parameter |
| Sequential pipeline | `SequentialAgent` |
| Parallel fan-out | `ParallelAgent` |
| Iterative loop | `LoopAgent` |
| LLM-driven routing | Automatic via sub-agent `description` |

### Demo 1 Architecture

```
Router Agent (no tools -- only routes)
  |-- billing_agent    -> lookup_invoice, process_refund
  |-- technical_agent  -> search_knowledge_base, check_system_status
  |-- escalation_agent -> create_escalation_ticket
```


In [ ]:
# Install dependencies (run once)
import subprocess, sys
subprocess.check_call([sys.executable, "-m", "pip", "install",
    "google-adk", "google-genai", "python-dotenv", "--ignore-installed", "cffi", "-q"])

In [ ]:
import os
from dotenv import load_dotenv
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

load_dotenv()
MODEL = "gemini-2.5-flash"
print("Ready" if os.getenv("GOOGLE_API_KEY") else "WARNING: Set GOOGLE_API_KEY in .env")

### Helper Function


In [ ]:
async def run_agent(agent, message):
    """Run an ADK agent and return the response."""
    service = InMemorySessionService()
    runner = Runner(agent=agent, app_name="demo", session_service=service)
    session = await service.create_session(app_name="demo", user_id="user1")
    content = types.Content(role="user", parts=[types.Part(text=message)])
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response() and event.content and event.content.parts:
            return event.content.parts[0].text
    return "(no response)"


### Step 1: Define Tools

Tools are plain Python functions. The LLM reads docstrings to decide when to call them.


In [ ]:
# Billing tools

def lookup_invoice(customer_email: str) -> dict:
    """Look up the most recent invoice for a customer by email."""
    invoices = {
        "jane@example.com": {"invoice_id": "INV-2024-001", "customer": "Jane Doe", "amount": "$99.00", "status": "paid", "plan": "Pro Monthly"},
        "bob@example.com": {"invoice_id": "INV-2024-002", "customer": "Bob Smith", "amount": "$299.00", "status": "overdue", "plan": "Enterprise Annual"},
        "alice@example.com": {"invoice_id": "INV-2024-003", "customer": "Alice Johnson", "amount": "$49.00", "status": "paid", "plan": "Starter Monthly"},
    }
    return invoices.get(customer_email.lower(), {"error": f"No invoice found for {customer_email}"})

def process_refund(invoice_id: str, reason: str) -> dict:
    """Process a refund for a specific invoice."""
    return {"refund_id": f"REF-{invoice_id[-3:]}", "status": "approved", "message": f"Refund for {invoice_id} approved. 5-7 business days."}

# Technical tools

def search_knowledge_base(query: str) -> dict:
    """Search the knowledge base for technical solutions."""
    articles = {
        "login": {"title": "Login Issues", "solution": "1. Clear cache. 2. Try incognito. 3. Reset password."},
        "crash": {"title": "App Crashing", "solution": "1. Update to v3.2.1. 2. Clear app data. 3. Check OS requirements."},
        "slow": {"title": "Performance Issues", "solution": "1. Check internet. 2. Close other apps. 3. Enable hardware acceleration."},
    }
    for keyword, article in articles.items():
        if keyword in query.lower():
            return article
    return {"title": "General Support", "solution": "No specific article found."}

def check_system_status() -> dict:
    """Check current status of all platform services."""
    return {"overall": "operational", "auth_service": "degraded", "last_incident": "2026-02-08"}

# Escalation tool

def create_escalation_ticket(customer_email: str, issue_summary: str, priority: str) -> dict:
    """Create an escalation ticket for issues needing human review."""
    times = {"low": "48 hours", "medium": "24 hours", "high": "4 hours", "critical": "1 hour"}
    return {"ticket_id": "ESC-2026-0042", "priority": priority, "estimated_response": times.get(priority, "24 hours")}


### Step 2: Create Agents

Each agent: name, description (used by router), instruction, and tools.


In [ ]:
billing_agent = Agent(
    name="billing_agent", model=MODEL,
    description="Handles billing: invoices, payments, refunds.",
    instruction="You are a billing specialist. Use lookup_invoice and process_refund.",
    tools=[lookup_invoice, process_refund],
)

technical_agent = Agent(
    name="technical_agent", model=MODEL,
    description="Handles technical issues: bugs, crashes, performance.",
    instruction="You are a technical specialist. Use search_knowledge_base and check_system_status.",
    tools=[search_knowledge_base, check_system_status],
)

escalation_agent = Agent(
    name="escalation_agent", model=MODEL,
    description="Handles complaints, disputes, security concerns.",
    instruction="You are an escalation specialist. Use create_escalation_ticket.",
    tools=[create_escalation_ticket],
)


### Step 3: Create the Router

The router has **no tools**. It reads each sub-agent's `description` to decide who handles each query.


In [ ]:
root_agent = Agent(
    name="customer_support_router", model=MODEL,
    instruction="Route to billing_agent, technical_agent, or escalation_agent. Never answer directly.",
    sub_agents=[billing_agent, technical_agent, escalation_agent],
)


### Step 4: Test It


In [ ]:
# Test 1: Billing
response = await run_agent(root_agent,
    "I need to check my invoice. Email: bob@example.com. Can I get a refund?")
print(response)


In [ ]:
# Test 2: Technical
response = await run_agent(root_agent,
    "My app keeps crashing when I login. Is there an outage?")
print(response)


In [ ]:
# Test 3: Escalation
response = await run_agent(root_agent,
    "Someone hacked my account! I see charges I didn't make. Email: jane@example.com. Urgent!")
print(response)


---
# Section 4: MCP -- Demo 2

**MCP (Model Context Protocol)** -- open standard for agents to connect to external tools/data at runtime.

```
Agent -> MCP Client (ADK) -> MCP Server (Supabase) -> Postgres DB
```

The agent **discovers tools at runtime** -- no hardcoded SQL or table definitions.

### Key Code

```python
supabase_mcp = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=["-y", "@supabase/mcp-server-supabase@latest",
                  "--access-token", TOKEN, "--project-ref", REF],
        ),
    ),
)

billing_agent = Agent(name="billing_mcp", model=MODEL, tools=[supabase_mcp])
```

### Run It

```bash
python demo2_mcp.py
```

### MCP vs Hardcoded Tools

| | Demo 1 (Hardcoded) | Demo 2 (MCP) |
|---|---|---|
| Data | Fake dictionaries | Real Supabase DB |
| Tools | Manual function defs | Auto-discovered at runtime |
| Queries | Fixed in code | Agent writes its own SQL |


---
# Section 5: A2A -- Demo 3

**A2A (Agent-to-Agent Protocol)** -- open standard for agents to communicate across network boundaries.

MCP connects agents to **tools**. A2A connects agents to **other agents**.

```
Your System                          Remote Service
  Root Agent  ---A2A (HTTP)--->  Shipping Agent (:8001)
    |-- Billing (MCP)                get_status()
    |-- Technical (local)            get_delivery()
    |-- Shipping (remote) ----^
```

### Key Code

```python
# Server side: expose agent via A2A
app = to_a2a(shipping_agent, port=8001)

# Client side: connect to remote agent
shipping = RemoteA2aAgent(name="shipping", agent_card="http://localhost:8001")
```

| Component | Purpose |
|-----------|---------|
| `to_a2a(agent)` | Expose agent as A2A service |
| `RemoteA2aAgent` | Connect to a remote A2A agent |
| Agent Card | JSON metadata at `/.well-known/agent-card.json` |


### Run It (two terminals)

**Terminal 1:** `uvicorn shipping_agent:app --port 8001`

**Terminal 2:** `python demo3_full_system.py`

### What Happens

1. Root agent receives query, recognizes it as shipping, delegates to `shipping_agent`
2. `RemoteA2aAgent` forwards request over HTTP to `localhost:8001`
3. Remote agent processes with its own tools, returns response via A2A

The router has **no idea** how the shipping agent works internally -- just its description.


---
# Full System Architecture

All three layers combined: routing + MCP + A2A.

```
        Router Agent
       /      |       \
  Billing   Technical  Shipping
   (MCP)    (Local)     (A2A)
     |        |           |
  Supabase  KB/Status  Remote :8001
```


### Run It

**Terminal 1:** `uvicorn shipping_agent:app --port 8001`

**Terminal 2:** `python demo3_full_system.py`

Tests three paths: Billing (MCP -> Supabase), Technical (local tools), Shipping (A2A -> remote).

### Interactive Demo

Run the Streamlit app for a visual, interactive version:

```bash
streamlit run streamlit_app.py
```


### Project Files

| File | What | Run |
|------|------|-----|
| `demo1_routing.py` | Multi-agent routing | `python demo1_routing.py` |
| `demo2_mcp.py` | MCP + Supabase | `python demo2_mcp.py` |
| `demo3_full_system.py` | Full system | `python demo3_full_system.py` |
| `shipping_agent.py` | A2A remote agent | `uvicorn shipping_agent:app --port 8001` |
| `streamlit_app.py` | Interactive demo | `streamlit run streamlit_app.py` |

### ADK Dev UI

Debug agents visually: `adk web`

### Deployment Options

| Option | Best For |
|--------|----------|
| `adk web` | Development |
| `adk api_server` | Simple API |
| Cloud Run | Containerized production |
| Agent Engine (Vertex AI) | Managed Google Cloud |


---
# Section 6: Homework

Build your own multi-agent system using ADK.

### Requirements

1. 3+ specialist agents with distinct responsibilities
2. A root router agent that delegates
3. At least one MCP integration (Supabase, filesystem, etc.)
4. At least one A2A integration (remote agent)

### Suggested Domains

| Domain | Agents |
|--------|--------|
| E-commerce | Product search, orders, customer service, inventory |
| Healthcare | Symptom checker, appointments, prescriptions |
| DevOps | Log analyzer, deployment manager, incident responder |
| Travel | Flights, hotels, itinerary, local recommendations |

### Deliverables

1. Working code demonstrating the full system
2. At least one standalone A2A agent file
3. Brief writeup: architecture, which agents use MCP vs A2A, one challenge you solved

### Stretch Goals

- `SequentialAgent` pipeline (Draft -> Review -> Edit)
- `ParallelAgent` fan-out (search multiple sources)
- `LoopAgent` iterative refinement (repeat until quality met)

---

## Summary

| Section | Key Concept |
|---------|-------------|
| 1 | Agent = LLM + Tools + Loop |
| 2 | Split work, not prompts |
| 3 | `sub_agents` + `description` = automatic routing |
| 4 | `McpToolset` auto-discovers tools at runtime |
| 5 | `RemoteA2aAgent` + `to_a2a()` = cross-service agents |

```
Layer 1: Multi-Agent -> Split god-agent into specialists
Layer 2: MCP         -> Connect agents to real data sources
Layer 3: A2A         -> Connect agents to other agents
```

## Resources

- [ADK Docs](https://google.github.io/adk-docs/) | [ADK GitHub](https://github.com/google/adk-python)
- [MCP Spec](https://modelcontextprotocol.io/) | [A2A Protocol](https://github.com/google/a2a-protocol)
- [Supabase MCP Server](https://github.com/supabase/mcp-server-supabase)
